# PPO Implementation examples

In [2]:
!pip install torchvision
!pip install torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 2.5 MB/s eta 0:00:00a 0:00:01


In [113]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

In [61]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [62]:
data[0, 2, 2, 2]

tensor(0.4442)

In [63]:
labels.shape

torch.Size([1, 1000])

The model is trained on single image, [1], [3] channels, 64 by 64 size and outputs 1000 classes. 

In [64]:
prediction = model(data) # forward pass

The calculated loss of the fisrt prediction is now backpropagated from the output to the input correcting the coeff

In [44]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

In [46]:
loss

tensor(-493.5576, grad_fn=<SumBackward0>)

Optimization is then loaded

In [114]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [51]:
# running optimization step each param is adjusted using gradient descent
optim.step() #gradient descent

In [52]:
loss

tensor(-493.5576, grad_fn=<SumBackward0>)

In [58]:
optim.zero_grad()

In [116]:
for i in range(1000):
    prediction = model(data)
    
    # loss = (prediction - labels).sum()
    loss = nn.CrossEntropyLoss()(prediction, labels)
    loss.backward() # backward pass
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # clip the gradient

    optim.step()
    optim.zero_grad()

    if i % 100 == 0:
        print(loss)

tensor(3328.5808, grad_fn=<DivBackward1>)
tensor(3344.7114, grad_fn=<DivBackward1>)
tensor(3329.3755, grad_fn=<DivBackward1>)
tensor(3328.8394, grad_fn=<DivBackward1>)
tensor(3328.7205, grad_fn=<DivBackward1>)
tensor(3328.6655, grad_fn=<DivBackward1>)
tensor(3328.6357, grad_fn=<DivBackward1>)
tensor(3328.6172, grad_fn=<DivBackward1>)


KeyboardInterrupt: 

In [83]:
prediction[0][:10]

tensor([-46.4438, -46.5104, -46.8415, -48.3938, -46.5488, -46.0211, -46.6106,
        -44.4758, -44.8752, -48.0595], grad_fn=<SliceBackward0>)

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input))
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2))
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2))
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2)
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1)
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4))
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5))
        # Gaussian layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6)
        return output


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [91]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [94]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0234, -0.0038,  0.1238,  0.0780, -0.0309, -0.0235, -0.0006,  0.0865,
          0.1221,  0.0133]], grad_fn=<AddmmBackward0>)


In [95]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [117]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4152, grad_fn=<MseLossBackward0>)


In [118]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [119]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0241, -0.0026, -0.0312, -0.0028, -0.0139,  0.0090])


In [121]:
# updating the weights of the network
# weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)